In [7]:
# 31000/4

In [8]:
# 7750.0 * 100


In [5]:
# 775000/5000 

In [6]:
# 155*3/24

In [7]:
# import pandas as pd
# import numpy as np
# import os
# !pip install nibabel

In [8]:
# csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"

In [9]:
# df = pd.read_csv(csv_path)
# print(df.shape)
# print(df.columns[:4],df.columns[-10:])
# df.head(2)

In [10]:
# df[df["MRN"]=="1093523"]

In [17]:
root_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/"
saving_path = "/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/"
import os
import nibabel as nib
import numpy as np
import pandas as pd
csv_path = "/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/pyradiomics_sybil_demog_CV.csv"
df = pd.read_csv(csv_path)
def which_address_exist(add1, add2):
    if os.path.exists(add1):
        return add1
    elif os.path.exists(add2):
        return add2
    else:
        return None

def get_base_name(filename):
    """Remove .nii.gz or .nii from filename cleanly."""
    if filename.endswith(".nii.gz"):
        return filename[:-7]
    elif filename.endswith(".nii"):
        return filename[:-4]
    else:
        return os.path.splitext(filename)[0]

for mrn in set(df["MRN"].to_list()):
    root_mrn = os.path.join(root_path, mrn+"_D")
    if os.path.exists(root_mrn):
        dates = df[df["MRN"]==mrn ]["date"].to_list()
        dates = [str(date) for date in dates]
        #print(dates)
        for date in dates:
            add1 = os.path.join(root_mrn, date)
            add2 = os.path.join(root_mrn, date+"_")
            root_mrn_date = which_address_exist(add1, add2)
            if root_mrn_date is None:
                continue
            df_mrn = df[(df["MRN"] == mrn) & (df["date"] == int(date))]
            info = df_mrn[['date', 'Date_To_Cancer', '1_year_risk', '2_year_risk', '3_year_risk', '4_year_risk','5_year_risk', '6_year_risk', 'age at dx', 'Sex', 'Race','Tobacco Use']]
            info_dict = info.iloc[0].to_dict()
            #print(df_mrn)
            #print(info)
            #print(info_dict)
            #print(mrn, root_mrn_date)
            ct_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and not any(x in i.lower() for x in ["_p", "seg", "sybil"])]
            seg_ = [i for i in os.listdir(root_mrn_date) if i.endswith(".nii.gz") and "_p" in i.lower()]
            if len(ct_)!=1 and len(seg_)!=1:
                print(len(ct_), len(seg_))
                continue
            ct = os.path.join(root_mrn_date, ct_[0])
            seg = os.path.join(root_mrn_date, seg_[0])
            #print(ct)
            #print(seg)
            ct_w = nib.load(ct)
            seg_w = nib.load(ct)
            # Check number of slices (assumes axial slices along third dimension)
            if ct_w.shape != seg_w.shape:
                print("Shape mismatch:", ct_w.shape, seg_w.shape, "→ Skipping", mrn, date)
                continue

            spacing_ct = ct_w.header.get_zooms()
            spacing_seg = seg_w.header.get_zooms()

            base_name = get_base_name(ct_[0])
            # Save data
            out_path = os.path.join(saving_path, base_name + ".npz")
            print("saving in ", out_path)
            np.savez(
                out_path,
                spacing_ct=spacing_ct,
                ct=ct_w.get_fdata(),  # native dtype
                seg=seg_w.get_fdata(),  # native dtype
                **info_dict
            )
        print("Saved:", out_path)
        print()

saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1263077_180731__0.0__.npz
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1263077_160328__2.34__.npz
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1263077_180213__0.46__.npz
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1263077_170919__0.86__.npz
Saved: /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/1263077_170919__0.86__.npz

saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/2455935_190320__0.87__.npz
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/2455935_200130__0.0__.npz
saving in  /rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/2455935_190703__

In [4]:
import numpy as np
a = np.load("/rsrch7/home/ip_rsrch/wulab/Mori/non_smoker_May/420022_160315__3.11__.nii.npz")

In [6]:
a = np.load("/rsrch7/home/ip_rsrch/wulab/Lung_Foundation_Model_Data_/Down-stream_tasks/NS_forFineTune/420022_160315__3.11__.npz")

In [7]:
for i in a.keys():
    print(i)

spacing_ct
ct
seg
date
Date_To_Cancer
1_year_risk
2_year_risk
3_year_risk
4_year_risk
5_year_risk
6_year_risk
age at dx
Sex
Race
Tobacco Use


In [9]:
a["ct"].shape

(512, 512, 140)

In [11]:
a["seg"].shape

(512, 512, 140)

In [12]:
a["ct"].dtype

dtype('float64')

In [13]:
a["seg"].dtype

dtype('float64')

In [9]:
import numpy as np
spacing_ct = np.array([0.7, 0.7, 2.0])
ct = np.random.randint(-1000, 200, (512, 512, 140), dtype=np.int16)
seg = np.zeros((512, 512, 140), dtype=np.uint8)
seg[200:220, 200:220, 60:65] = 1  # small cuboid region set to 1
age = 63
Sex = "Male"
Race = "Asian"
histology_label = 1  # ← squamous
histology_adc = 0
histology_squ = 1
histology_smc = 0
np.savez("synthetic_lung_case.npz",
         spacing_ct=spacing_ct,
         ct=ct,
         seg=seg,
         age=age,
         Sex=Sex,
         Race=Race,
         histology_label=histology_label,
         histology_adc=histology_adc,
         histology_squ=histology_squ,
         histology_smc=histology_smc)

In [10]:
data = np.load("synthetic_lung_case.npz")

# Access elements
ct = data["ct"]
seg = data["seg"]
spacing_ct = data["spacing_ct"]
age = int(data["age"])
Sex = str(data["Sex"])
Race = str(data["Race"])

print("CT shape:", ct.shape)
print("Seg shape:", seg.shape, "Unique labels:", np.unique(seg))
print("Spacing:", spacing_ct)
print("Age:", age)
print("Sex:", Sex)
print("Race:", Race)


CT shape: (512, 512, 140)
Seg shape: (512, 512, 140) Unique labels: [0 1]
Spacing: [0.7 0.7 2. ]
Age: 63
Sex: Male
Race: Asian
